In [39]:
import pandas as pd
import seaborn as sns
#pip install openpyxl 

#### Pregunta N ° 1 - Objetivos:

Elaborar una tabla con la facturación de cada producto considerando la zona.  

In [40]:
# Cargamos como un DataFrame (DF) los datos a analizar

df = pd.read_excel("FACT_TOTAL.XLSX", sheet_name = "BASE")
df

# Cargamos el Excel donde iran las respuestas
##pregunta1 = pd.read_excel("pregunta1.xlsx")

,CODIGO_ZONA,CODIGO_DE_COMPRADOR,CANTIDAD_PEDIDA,CANTIDAD_DESPACHADA,CODIGO_DEL_PRODUCTO
0,807,633115,2,2,283760
1,807,633115,3,2,283760
2,810,309373,2,2,283760
3,810,309373,3,3,296648
4,810,309373,2,2,306259
...,...,...,...,...,...
169300,619,113156,2,2,283760
169301,619,113156,2,2,283760
169302,619,113156,2,1,592959
169303,619,113156,4,3,707645


#### Analisis general


In [41]:
## Declaracion de variables

id_zona = df['CODIGO_ZONA']
id_comprador = df['CODIGO_DE_COMPRADOR']
cantidad_pedida = df['CANTIDAD_PEDIDA']
cantidad_despachada = df['CANTIDAD_DESPACHADA']
id_producto = df['CODIGO_DEL_PRODUCTO']

In [ ]:
## Informacion sobre las columnas

df.info() # los ids de los productos son de tipo objeto. Se procede a analizar los datos de la columna.

In [ ]:
## Analisis estadistico 

df.describe()

In [ ]:
## Valores de las distintas columnas

print(set(id_producto)) # (!) tiene caracteres entre los numeros
print(set(id_zona)) # ok
print(set(cantidad_pedida)) # ok
print(set(cantidad_despachada)) #ok

## Ver cuantas zonas y cuantos productos hay

print(len(set(id_zona))) #163 Zonas de reparto unicas
print(len(set(id_producto))) #801 Productos unicos

#### Eliminar caracteres y convertir los ids de producto en enteros

In [45]:
id_producto.replace({'X':'','V':''}, regex=True, inplace=True)

In [ ]:
set(id_producto)

In [47]:
df = df.astype({"CODIGO_DEL_PRODUCTO" : int})

In [ ]:
df.dtypes

#### Preparar los datos

In [ ]:
ventas_por_producto_y_zona = pd.DataFrame(df.groupby(['CODIGO_DEL_PRODUCTO', 'CODIGO_ZONA'])['CANTIDAD_PEDIDA'].sum().sort_values(ascending = False))
ventas_por_producto_y_zona

In [50]:
despachos_por_producto_y_zona = pd.DataFrame(df.groupby(['CODIGO_DEL_PRODUCTO', 'CODIGO_ZONA'])['CANTIDAD_DESPACHADA'].sum().sort_values(ascending = False))
despachos_por_producto_y_zona.sort_values('CODIGO_ZONA')

,,CANTIDAD_DESPACHADA
CODIGO_DEL_PRODUCTO,CODIGO_ZONA,
584511,101,5
147213,101,6
829010,101,2
39633,101,6
299808,101,22
...,...,...
861703,828,1
504969,828,3
234196,828,2


In [ ]:
compradores_por_zona = pd.DataFrame(df.groupby(['CODIGO_ZONA'])['CODIGO_DE_COMPRADOR'].nunique())
compradores_por_zona

In [57]:
compradores_por_zona.rename({'CODIGO_DE_COMPRADOR': 'NUM_COMPRADORES'},axis=1,inplace=True)
compradores_por_zona

,NUM_COMPRADORES
CODIGO_ZONA,
101,472
102,359
103,590
104,452
107,400
...,...
824,8
825,1
826,1


In [53]:
compradores_por_zona.to_excel('p1-num_compradores.xlsx')

In [ ]:
ventas_por_producto_y_zona["CANTIDAD_DESPACHADA"] = despachos_por_producto_y_zona["CANTIDAD_DESPACHADA"]
ventas_por_producto_y_zona.sort_values('CODIGO_DEL_PRODUCTO')
ventas_por_producto_y_zona = df
df

In [55]:
df["Variacion pedido-despacho"] = (df["CANTIDAD_DESPACHADA"] - df["CANTIDAD_PEDIDA"])/(df["CANTIDAD_PEDIDA"])*100
df

,CODIGO_ZONA,CODIGO_DE_COMPRADOR,CANTIDAD_PEDIDA,CANTIDAD_DESPACHADA,CODIGO_DEL_PRODUCTO,Variacion pedido-despacho
0,807,633115,2,2,283760,0.000000
1,807,633115,3,2,283760,-33.333333
2,810,309373,2,2,283760,0.000000
3,810,309373,3,3,296648,0.000000
4,810,309373,2,2,306259,0.000000
...,...,...,...,...,...,...
169300,619,113156,2,2,283760,0.000000
169301,619,113156,2,2,283760,0.000000
169302,619,113156,2,1,592959,-50.000000
169303,619,113156,4,3,707645,-25.000000


In [56]:
df.to_excel('p1-pedidos-despachos.xlsx')